# Writing questions about Clinvar

Goal: Evaluate RAG

## Data preparation

In [ ]:
import pandas as pd
import json
from pprint import pprint
import polars as pl
import random


df = pl.read_csv("../../data/mitocarta/Human.MitoCarta3.0.csv", sep=",")

Final structure 
                               _key human_gene_id  symbol  \
_created_at                                                
2025-10-13T07:41:37.610231Z   1537          1537    CYC1   
2025-10-13T07:41:37.610356Z   6390          6390    SDHB   
2025-10-13T07:41:37.610422Z  10229         10229    COQ7   
2025-10-13T07:41:37.610708Z   6389          6389    SDHA   
2025-10-13T07:41:37.611244Z   7384          7384  UQCRC1   

                                                             synonyms  \
_created_at                                                             
2025-10-13T07:41:37.610231Z                           [MC3DN6, UQCR4]   
2025-10-13T07:41:37.610356Z  [CWS2, IP, PGL4, SDH, SDH1, SDH2, SDHIP]   
2025-10-13T07:41:37.610422Z              [CAT5, CLK-1, CLK1, COQ10D8]   
2025-10-13T07:41:37.610708Z      [CMD1GG, FP, PGL5, SDH1, SDH2, SDHF]   
2025-10-13T07:41:37.611244Z                    [D3S3191, QCR1, UQCR1]   

                                                 

In [ ]:
import pandas as pd

def expand_info_column(df: pd.DataFrame, info_col: str = "info") -> pd.DataFrame:
    """
    Aplati la colonne 'info' d'un DataFrame contenant des dictionnaires variables.
    Chaque clé rencontrée dans 'info' devient une colonne distincte.
    """
    # Étape 1 : Identifier toutes les clés présentes dans 'info'
    all_info_keys = set()
    for entry in df[info_col]:
        if isinstance(entry, dict):
            all_info_keys.update(entry.keys())
    
    # Étape 2 : Créer un DataFrame à partir des dicts
    info_expanded = pd.DataFrame([
        {k: entry.get(k, None) if isinstance(entry, dict) else None for k in all_info_keys}
        for entry in df[info_col]
    ])
    
    # Étape 3 : Préfixer les colonnes pour clarté
    info_expanded = info_expanded.add_prefix(f"{info_col}.")
    
    # 💡 Étape 4 : Réinitialiser les index pour éviter l’erreur de concaténation
    df_reset = df.reset_index(drop=True)
    info_expanded_reset = info_expanded.reset_index(drop=True)
    
    # Étape 5 : Combiner avec le DataFrame d’origine (sans la colonne 'info')
    df_flat = pd.concat([df_reset.drop(columns=[info_col]), info_expanded_reset], axis=1)
    
    return df_flat


df_flat = expand_info_column(df, info_col="info")
df_flat.head()
df_flat.columns

df = pl.from_pandas(df_flat)